In [1]:
import os

In [2]:
%pwd

'f:\\OneDrive - MSFT\\wine_quality_ML\\Machine-Learning-End-to-End-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\OneDrive - MSFT\\wine_quality_ML\\Machine-Learning-End-to-End-Project'

In [5]:
# entity class
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvalutionConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: str
    target_column: str

In [6]:
from MLProject_WineQT.constants.const import *
from src.MLProject_WineQT.utils.common import read_yaml, create_directories,save_json
import pandas as pd

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        # Load config, params, and schema YAML files
        self.config = read_yaml(config_filepath)  # Ensure this loads the YAML correctly
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Debugging: Print loaded config, params, and schema
        print(f"Loaded config: {self.config}")
        print(f"Loaded params: {self.params}")
        print(f"Loaded schema: {self.schema}")

        # Create directories specified in the config
        create_directories([self.config.artifacts_root])
        
    def get_model_evalution_config(self) -> ModelEvalutionConfig:
        config = self.config.model_evalution
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        
        
        model_evalution_config = ModelEvalutionConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name
        )
        return model_evalution_config

In [8]:
# component
import os
import pandas as pd
import json
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import numpy as np
import joblib


In [9]:
class ModelEvalution:
    def __init__(self,config:ModelEvalutionConfig):
        self.config = config 
        
    def eval_metrics(self, actual, pred):
            rmse = np.sqrt(mean_squared_error(actual,pred))
            mae = mean_absolute_error(actual,pred)
            r2 = r2_score(actual,pred)
        
            return rmse,mae,r2
    
    def save_result(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)
        (rmse,mae,r2)=self.eval_metrics(test_y, predicted_qualities)
        score = {'rmse':rmse,'mae':mae,'r2':r2}
        save_json(path=Path(self.config.metric_file_name),data=score)

In [10]:
#pipeline
try:
    config = ConfigurationManager()
    model_evalution_config = config.get_model_evalution_config()
    model_evalution_config = ModelEvalution(config=model_evalution_config)
    model_evalution_config.save_result()
except Exception as e:
    raise e

[2024-10-05 14:01:37,179: INFO: common: yaml file:config\config.yaml load successfuly]
[2024-10-05 14:01:37,184: INFO: common: yaml file:params.yaml load successfuly]
[2024-10-05 14:01:37,191: INFO: common: yaml file:schema.yaml load successfuly]
Loaded config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/AliTheAnalyst01/livesensor/raw/refs/heads/main/WineQT.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion/unzipped_data'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'unzip_data_dir': 'artifacts/data_ingestion/unzipped_data/WineQT.csv'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/unzipped_data/WineQT.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv'